In [ ]:
import re
import os
from typing import Iterator, DefaultDict, Dict
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fasta_parser import parse_fasta

In [ ]:
filename1 = 'Results/Length/All_Chromosomes_length.csv'
genome_len = pd.read_csv(filename1, names=['name', 'len'])

In [ ]:
def genome_length(filename, name):
    genome_len = pd.read_csv(filename1, names=['name', 'len'])
    gen_len = genome_len['len'].loc[genome_len['name'] == name].values[0]
    return gen_len

In [ ]:
gen_len = genome_length(filename1, 'Actinotignum')

In [ ]:
filename = 'Results/kmer_counts/Actinotignum/GCF_000724605.1_chr_k2_8_chr.csv'

In [ ]:
def get_kmer_frequencies_csvs(filename, seq_len):
    df_grp = []
    km_data = pd.read_csv(filename, names=['kmer', 'count'])
    km_data['length'] = km_data['kmer'].str.len()
    km_grouped = km_data.groupby('length')
    for i in (range(2, 9)):
        grp = km_grouped.get_group(i).copy()
        grp['freq'] = grp['count'] / (seq_len - i + 1)
        print(f"Frequency of kmers of lenth {i}: {grp['freq'].sum()}")
        df_grp.append(grp)
    return pd.concat(df_grp)

In [ ]:
get_kmer_frequencies_csvs(filename, gen_len)

Frequency of kmers of lenth 2: 1.0
Frequency of kmers of lenth 3: 1.0
Frequency of kmers of lenth 4: 1.0
Frequency of kmers of lenth 5: 1.0
Frequency of kmers of lenth 6: 1.0
Frequency of kmers of lenth 7: 1.0
Frequency of kmers of lenth 8: 1.0


,kmer,count,length,freq
0,AA,95782,2,0.044358
1,AC,105375,2,0.048800
2,AG,106131,2,0.049151
3,AT,100621,2,0.046599
4,CA,122154,2,0.056571
...,...,...,...,...
87371,TTTTTTGT,12,8,0.000006
87372,TTTTTTTA,4,8,0.000002
87373,TTTTTTTC,5,8,0.000002
87374,TTTTTTTG,7,8,0.000003


In [ ]:
df_grp = []
for i in (range(2, 9)):
    grp = km_grouped.get_group(i).copy()
    grp['freq'] = grp['count'] / (gen_len - i + 1)
    print(f"Frequency of kmers of lenth {i}: {grp['freq'].sum()}")
    df_grp.append(grp)
df_final = pd.concat(df_grp)

In [ ]:
df_final

In [ ]:
km_data = pd.read_csv(filename, names=['kmer', 'count'])
km_data['length'] = km_data['kmer'].str.len()
km_grouped = km_data_count.groupby('length')

In [ ]:
def get_kmer_stats_genome(filename, name, gen_len):
    name = filename.split('/')[2]
    km_data = pd.read_csv(filename, names=['kmer', 'count'])
    km_data['length'] = km_data['kmer'].str.len()
    km_data_count = km_data.loc[km_data['count'] != 0]
    km_total = km_data_count.groupby('length')['kmer'].count()
    kmer_cnt_by_k = pd.DataFrame(km_total).reset_index().rename(columns={'length': 'k', 'kmer': 'num'})
    k_list = kmer_cnt_by_k['k'].to_list()
    ks = f'{k_list[0]}-{k_list[-1]}'
    num_kmer_found = kmer_cnt_by_k['num'].to_list()
    gen_len = genome_len['len'].loc[genome_len['name'] == name].values[0]
    positions = [(gen_len - x + 1) for x in k_list]
    all_kmers = [(4**x) for  x in k_list]
    print(f'Summary statistics of kmer counts in the genus {name}')
    print(f'The kmer length testeds {ks}')
    print(f'Genome length {gen_len}\n')
    print('k\tObs\t4^k\tN (kpb)\tNF\tRepeated')
    for i, k in enumerate(k_list):
        obs = num_kmer_found[i]
        pos = 4**k
        n = (positions[i]-k) + 1 
        m = all_kmers[i] - num_kmer_found[i]
        r = positions[i] - num_kmer_found[i]
        print(f'{k}\t{obs}\t{pos}\t{n}\t{m}\t{r}')

In [ ]:
get_kmer_stats_genome(filename, 'Actinotignum', gen_len)

Summary statistics of kmer counts in the genus Actinotignum
The kmer length testeds 2-8
Genome length 2159306

k	Obs	4^k	N (kpb)	NF	Repeated
2	16	16	2159304	0	2159289
3	64	64	2159302	0	2159240
4	256	256	2159300	0	2159047
5	1024	1024	2159298	0	2158278
6	4096	4096	2159296	0	2155205
7	16384	16384	2159294	0	2142916
8	65291	65536	2159292	245	2094008


In [ ]:
# Summary statistics of kmer counts in the genus Actinotignum
# The kmer length testeds 2-8
# Genome length 2159306

# k	Obs	4^k	N (kpb)	NF	Repeated
# 2	16	16	2159304	0	2159289
# 3	64	64	2159302	0	2159240
# 4	256	256	2159300	0	2159047
# 5	1024	1024	2159298	0	2158278
# 6	4096	4096	2159296	0	2155205
# 7	16384	16384	2159294	0	2142916
# 8	65291	65536	2159292	245	2094008

In [ ]:
def prob_kmer_qual_dist(k):
    return 1/4**k

In [ ]:
prob_kmer_qual_dist(3)

0.015625

### GC content

In [ ]:
ffasta = 'Data/Genomes_splitted/Thermodesulfatator/Chromosomes/GCF_000217795.1_chr.fna.gz'

for Id, sequence in parse_fasta(ffasta):
    s = sequence

NameError: ignored

In [ ]:
def get_sequence_chunks(sequence, window, step, as_overlap=False):
    """
    GC Content in a DNA/RNA sub-sequence length k. 
    In overlap windows of length k, other wise non overlapping.

    Inputs:
        sequence - a string representing a DNA sequence.
        as_overlap - boolean that represents if overlap is needed.
        k - a integer representing the lengths of overlapping bases.
    Outputs:
        subseq - a string representing a slice of the sequence with or without
                 overlapping characters.
    """
    sequence = sequence.upper()

    if as_overlap:
        # overlap sequence length
        step = 1

    # iterates to the overlap region
    for i in range(0, len(sequence) - window + 1, step):
        # creates the substring
        subseq = sequence[i:i + window]
        start = i
        end = i + window
        yield subseq, start, end

In [ ]:
list(get_sequence_chunks(s, 2000, 500, as_overlap=True))

In [ ]:
def gc_content(seq: str, percent: bool = True) -> float:
    """
    Calculate G+C content, return percentage (as float between 0 and 100).
    Copes mixed case sequences, and with the ambiguous nucleotide S (G or C)
    when counting the G and C content.  The percentage is calculated against
    the full length.

    Inputs:
        sequence - a string representing a sequence (DNA/RNA/Protein)

    Outputs:
        gc - a float number or a percent value representing the gc content of the sequence.

    """
    seq_len = len(seq)
    gc = sum(1 for x in seq if x.upper() in {"G", "C", "S"})
    if seq_len == 0:
        return 0
    gc /= seq_len
    if percent:
        return gc * 100
    return gc

In [ ]:
%timeit gc_content('CATGGGCATCGGCCATACGCC')

In [ ]:
gc_total = gc_content('CATGGGCATCGGCCATACGCC')

In [ ]:
def get_sequence_skew(sequence):
    """Returns the difference between the total number of 
    occurrences of G and the total number of occurrences of C in 
    the first i elements of the sequence. """
    skew = [0]
    for idx, element in enumerate(sequence):
        if sequence[idx] == 'G':
            skew.append(skew[idx] + 1)
        elif sequence[idx] == 'C':
            skew.append(skew[idx] -1)
        else:
            skew.append(skew[idx])
    return skew

In [ ]:
seq_skw = get_sequence_skew('CATGGGCATCGGCCATACGCC')

In [ ]:
pd.DataFrame(seq_skw, columns=['seq1']).plot()

In [ ]:
import glob

In [ ]:
glob.glob(f'Data/Genomes_splitted/*/Chromosomes/*_chr.fna.gz')

In [ ]:
seqs = ['CATGGGCATCGGCCATACGCC', 'CGTGGGCATCGGCCACACGCC', 'CATAGACATCGGCCATACGCCAATGC']

dfs = []
for idx, seq in enumerate(seqs):
    gc = get_sequence_skew(seq)
    dfs.append(pd.DataFrame(gc))
    
pd.concat(dfs   , axis=1, ignore_index=True).plot()

In [ ]:
plt.plot(get_sequence_skew('CATGGGCATCGGCCATACGCC'))

In [ ]:
plt.hist(get_sequence_skew('CATGGGCATCGGCCATACGCC'));

In [ ]:
def get_gc_content_slide_window(sequence: str, window: int, step: int) -> DefaultDict[str, float]:
    """
    Calculate the GC (G+C) content along a sequence. Returns a dictionary-like
    object mapping the sequence window to the gc value (floats).
    Returns 0 for windows without any G/C by handling zero division errors, and
    does NOT look at any ambiguous nucleotides.

    Inputs:
        sequence - a string representing a sequence (DNA/RNA/Protein)
        window_size - a integer representing the length of sequence to be analyzed
                   at each step in the full sequence.
        step - a integer representing the length of overlapping sequence allowed.

    Outputs:
        gc- a dictionary-like object mapping the window size sequence to it gc values
            as floating numbers
    """
    gc = defaultdict(float)
    for s, i, e in get_sequence_chunks(sequence, window, step, as_overlap=False):
        key = f'{i}-{e}'
        gc[key] = gc_content(s)
    return gc

In [ ]:
gc_slide = get_gc_content_slide_window('CATGGGCATCGGCCATACGCC', 10, 10)

In [ ]:
gc_slide

In [ ]:
df_gc_content_window = pd.DataFrame(gc_slide.items(), columns=['window', 'gc(%)'])

In [ ]:
df_gc_content_window

In [ ]:
# d_i = {
#         idx: gc_chunk - gc_total
#         for idx, data in gc_slide.items()
#         for chunk, gc_chunk in data.items()
#     }
# d_i

In [ ]:
def difference_gc(total_gc: float, gc_dict: Dict[str, float]) -> Dict[str, float]:
    """
    Calculates the difference between the mean GC content of window i, and the
    mean chromosomal GC content, as Di = GCi − GC
    """
    # iterate through all keys, gc calculated values for the
    # genome chunk
    # get the difference between the chromosomal mean and the chunks
    # add the difference to the appropriate chunk
    d_i = {
        chunk: gc_cnt - total_gc
        for chunk, gc_cnt in gc_dict.items()
    }
    return d_i

In [ ]:
dif_gc = difference_gc(gc_total, gc_slide)

In [ ]:
dif_gc

In [ ]:
df_gc_window = pd.DataFrame(dif_gc.items(), columns=['window', 'diff'])

In [ ]:
df_gc_window

In [ ]:
pd.merge(df_gc_content_window, df_gc_window, how= 'left', on='window')

In [ ]:
def get_chromosomal_gc_variation(difference_dic: Dict[str, float]) -> float:
    """
    Calculates the chromosomal GC variation defined as the log-transformed average
    of the absolute value of the difference between the mean GC content of each
    non-overlapping sliding window i and mean chromosomal GC content.
    chromosomal_gc_variantion = log(1/N*sum(|Di|), where N is the maximum number of
    non-overlapping window size in bp in a sliding windows.
    """
    # get the number of chunks
    n = len(difference_dic)
    arr = np.fromiter(
        difference_dic.values(),
        dtype=np.float32,
        count=len(difference_dic),
    )
    var = np.log(np.sum(np.abs(arr)) / n)
    return var

In [ ]:
gc_var = get_chromosomal_gc_variation(dif_gc)

In [ ]:
gc_var

In [ ]:
var_series = pd.Series(gc_var, index=['species1'])
gc_series = pd.Series(gc_total, index=['species1'])

In [ ]:
name = 'spec1'

In [ ]:
pd.DataFrame([var_series, 
              gc_series], 
             columns=['species1']).T.rename(columns={0: 'gc_var', 
                                                     1: 'gc_total'}).reset_index().rename(columns={'index': 'name'})

In [ ]:
def get_minimum_skew(sequence):
    """Returns a position in a sequence minimizing the skew."""
    min_skew = []
    skew = get_sequence_skew(sequence)
    m_skew = min(skew)
    for idx in range(len(sequence) + 1):
        if skew[idx] == m_skew:
            min_skew.append(idx)
    return min_skew

In [ ]:
get_minimum_skew('CATGGGCATCGGCCATACGCC')

In [ ]:
gc = re.findall('[gc]', 'CATGGGCATCGGCCATACGCC', re.IGNORECASE)
len(gc), len('CATGGGCATCGGCCATACGCC')

In [ ]:
gc_content('CATGGGCATCGGCCATACGCC', False)

In [ ]:
def gc(sequence):
    """ Return the GC content, record ID for a sequence """
    return (len(re.findall('[gc]', sequence, re.IGNORECASE)) / len(sequence)) * 100
#     return (gc / len_seq) * 100

In [ ]:
%timeit gc('CATGGGCATCGGCCATACGCC')

In [ ]:
def test_gc():
    """ Test gc """

    assert gc('') == 0.
    assert gc('C') == 100.
    assert gc('G') == 100.
    assert gc('CGCCG') == 100.
    assert gc('ATTAA') == 0.
    assert gc('ACGT') == 50.

In [ ]:
gc('ATTAA')

In [ ]:
sub_seq = 'GGC'
seq = 'CATGGGCATCGGCCATACGCC'

In [ ]:
print(*[m.start() + 1 for m in re.finditer(f'(?=({sub_seq}))', seq)])

In [ ]:
def get_sequence_chunks(sequence, window, as_overlap=False):
    """GC Content in a DNA/RNA sub-sequence length k. In
    overlapp windows of lenght k.
    
    Inputs:
    
        sequence - a string representing a DNA sequence.    
        as_overlap - boolean that represents if overlap is needed.
        k - a integer reppresenting the lengths of overlappig bases.
            Default is 20.
    
    Outputs:
    
        gc_content - an array-like object with 
    
    
    """
    # make sequence upper case and getting the length of it
    sequence, seq_len = sequence.upper(), len(sequence)
    # non overlap sequence length
    non_overlap = range(0, seq_len - window + 1, window)
    # overlap sequence length
    overlap = range(0, seq_len - window + 1)
    # overlap is needed
    if as_overlap:
        # iterates to the overlap region
        for i in overlap:
            # creates the substring 
            subseq = sequence[i:i + window]
            yield subseq
    # if non overlap is choosed
    else:
        # iterates to the mon overlap region
        for j in non_overlap:
            # creates the substring to count the gc_content
            subseq = sequence[j:j + window]
            yield subseq

In [ ]:
list(get_sequence_chunks('ACGTGGCCATACGT', 5, as_overlap=False))

In [ ]:
# def GC(seq, percent=True):
#     """
#     Calculate G+C content, return percentage (as float between 0 and 100).
#     Copes mixed case sequences, and with the ambiguous nucleotide S (G or C)
#     when counting the G and C content.  The percentage is calculated against
#     the full length. It looks for ambigous bases as S (C or G) or s (c or g).
    
#     Inputs:
#         sequence - a string representing a sequence (DNA/RNA/Protein)
    
#     Outputs:
#         gc - a float number or a percent value representing the gc content of the sequence.     
    
#     """
#     gc = sum(seq.count(x) for x in ["G", "C", "g", "c", "S", "s"])
#     if percent:
#         try:
#             return gc * 100.0 / len(seq)
#         except ZeroDivisionError:
#             return 0.0 
#     else:
#         return gc / len(seq)

In [ ]:
def gc_content_sequence_window(sequence, as_overlap=False, k=20):
    """GC Content in a DNA/RNA sub-sequence length k. In
    overlapp windows of lenght k.
    
    Inputs:
    
        sequence - a string representing a DNA sequence.    
        as_overlap - boolean that represents if overlap is needed.
        k - a integer reppresenting the lengths of overlappig bases.
            Default is 20.
    
    Outputs:
    
        gc_content - an array-like object with 
    
    
    """
    # make sequence upper case and getting the length of it
    sequence = sequence.upper()
    seq_len = len(sequence)
    # the array-like object to collect the data
    gc = []
    # non overlap sequence length
    non_overlap = range(0, len(sequence) - k + 1, k)
    # overlap sequence length
    overlap = range(0, seq_len - k + 1)
    # overlap is needed
    if as_overlap:
        # iterates to the overlap region
        for i in overlap:
            # creates the substring to count the gc_content
            subseq = sequence[i:i + k]
            # collect the data in the array container
            gc.append((i, gc_content(subseq, True)))
    # if non overlap is choosed
    else:
        # iterates to the mon overlap region
        for j in non_overlap:
            # creates the substring to count the gc_content
            subseq = sequence[j:j + k]
            # collect the data in the array container
            gc.append((j, gc_content(subseq, True)))
    return gc

In [ ]:
gc_content_sequence_window('ACGTGGCCATACGT', as_overlap=False, k=5)

In [ ]:
def gc_var(sequence, as_overlap=False, k=20):
    """
    Calculates the gc content variance in a sequence according to a 
    window of length k.
    
    Inputs:
        sequence - a string representing a DNA sequence.
        k - integer representing the length of the search window.
            default is 20.
    
    Outputs:
    
        log of the gc variantion in the sequence in a window space of
        length k.
    
    """
    # calculates the percent of gc content
    gc = gc_content(sequence, True)
    # get the gc content in the window space as an array
    gc_i = np.array(gc_content_sequence_window(sequence, as_overlap, k=k))
    # get the len of the gc content in the window space
    len_gc_i = np.shape(gc_i)[0]
    # check the difference of each point 
    dif = gc_i - gc
    return np.log((1 / len_gc_i) * sum(abs(dif)))

In [ ]:
gc_var('ACGTGGCCATACGT', as_overlap=False, k=5)

In [ ]:
list(get_sequence_chunks2('ACGTGGCCATACGT', 5, 1, as_overlap=False))